In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [15]:
#read data and remove any NaN
d=pd.read_csv("movie_metadata.csv")
d=d.dropna(axis=0, how='any')
display(d)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000.0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0.0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000.0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000.0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,Color,Olivier Assayas,81.0,110.0,107.0,45.0,Béatrice Dalle,576.0,136007.0,Drama|Music|Romance,...,39.0,French,France,R,4500.0,2004.0,133.0,6.9,2.35,171.0
5027,Color,Jafar Panahi,64.0,90.0,397.0,0.0,Nargess Mamizadeh,5.0,673780.0,Drama,...,26.0,Persian,Iran,Not Rated,10000.0,2000.0,0.0,7.5,1.85,697.0
5033,Color,Shane Carruth,143.0,77.0,291.0,8.0,David Sullivan,291.0,424760.0,Drama|Sci-Fi|Thriller,...,371.0,English,USA,PG-13,7000.0,2004.0,45.0,7.0,1.85,19000.0
5035,Color,Robert Rodriguez,56.0,81.0,0.0,6.0,Peter Marquardt,121.0,2040920.0,Action|Crime|Drama|Romance|Thriller,...,130.0,Spanish,USA,R,7000.0,1992.0,20.0,6.9,1.37,0.0


In [16]:
print(d.shape)

(3714, 28)


In [17]:
def classify(col):
    if col['imdb_score'] >= 0 and col['imdb_score'] < 4:
        return 0
    elif col['imdb_score'] >= 4 and col['imdb_score'] < 6:
        return 1
    elif col['imdb_score'] >= 6 and col['imdb_score'] < 7:
        return 2
    elif col['imdb_score'] >= 7 and col['imdb_score'] < 8:
        return 3
    elif col['imdb_score'] >= 8 and col['imdb_score'] <= 10:
        return 4

In [20]:
d['Success'] = d.apply(classify, axis=1)

In [21]:
d.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Success
count,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000,3.714000e+03,3.714000e+03,3714.000000,3714.000000,3714.000000,3.714000e+03,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000
mean,167.597738,110.194938,815.346257,773.124933,7751.467959,5.268190e+07,1.057848e+05,11537.025310,1.375067,337.253096,4.644696e+07,2002.993807,2027.511039,6.463543,2.112009,9398.466882,2.095854
std,123.687539,22.371908,3084.488244,1901.189298,15557.969368,7.035605e+07,1.521139e+05,19166.731777,2.043595,411.791222,2.272446e+08,9.840813,4555.515896,1.054684,0.353450,21556.398386,0.919882
min,2.000000,37.000000,0.000000,0.000000,0.000000,1.620000e+02,9.100000e+01,0.000000,0.000000,4.000000,2.180000e+02,1927.000000,0.000000,1.600000,1.180000,0.000000,0.000000
25%,77.000000,96.000000,11.000000,195.000000,745.000000,8.382841e+06,1.962575e+04,1933.250000,0.000000,110.000000,1.000000e+07,1999.000000,387.000000,5.900000,1.850000,0.000000,1.000000
50%,139.000000,106.000000,64.000000,436.000000,1000.000000,3.011697e+07,5.399350e+04,4073.000000,1.000000,210.000000,2.500000e+07,2004.000000,689.500000,6.600000,2.350000,226.500000,2.000000
75%,224.000000,120.000000,241.000000,691.750000,13000.000000,6.684870e+07,1.286200e+05,16261.000000,2.000000,398.750000,5.000000e+07,2010.000000,976.000000,7.200000,2.350000,11000.000000,3.000000
max,813.000000,325.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.300000,16.000000,349000.000000,4.000000


In [22]:
d.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Success
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000.0,3
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0.0,3
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000.0,2
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000.0,4
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000.0,2


In [26]:
def fill_nan(col):
    d[col] = d[col].fillna(d[col].median())

cols = list(d.columns)
fill_nan(cols)

In [27]:
d.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Success
count,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000,3.714000e+03,3.714000e+03,3714.000000,3714.000000,3714.000000,3.714000e+03,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000,3714.000000
mean,167.597738,110.194938,815.346257,773.124933,7751.467959,5.268190e+07,1.057848e+05,11537.025310,1.375067,337.253096,4.644696e+07,2002.993807,2027.511039,6.463543,2.112009,9398.466882,2.095854
std,123.687539,22.371908,3084.488244,1901.189298,15557.969368,7.035605e+07,1.521139e+05,19166.731777,2.043595,411.791222,2.272446e+08,9.840813,4555.515896,1.054684,0.353450,21556.398386,0.919882
min,2.000000,37.000000,0.000000,0.000000,0.000000,1.620000e+02,9.100000e+01,0.000000,0.000000,4.000000,2.180000e+02,1927.000000,0.000000,1.600000,1.180000,0.000000,0.000000
25%,77.000000,96.000000,11.000000,195.000000,745.000000,8.382841e+06,1.962575e+04,1933.250000,0.000000,110.000000,1.000000e+07,1999.000000,387.000000,5.900000,1.850000,0.000000,1.000000
50%,139.000000,106.000000,64.000000,436.000000,1000.000000,3.011697e+07,5.399350e+04,4073.000000,1.000000,210.000000,2.500000e+07,2004.000000,689.500000,6.600000,2.350000,226.500000,2.000000
75%,224.000000,120.000000,241.000000,691.750000,13000.000000,6.684870e+07,1.286200e+05,16261.000000,2.000000,398.750000,5.000000e+07,2010.000000,976.000000,7.200000,2.350000,11000.000000,3.000000
max,813.000000,325.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.300000,16.000000,349000.000000,4.000000


In [29]:
#Splitting and evzluating data
trainx,testx,trainy,testy=train_test_split(d[d.columns[5:32]],d.iloc[:,-1],test_size=0.3,random_state=0)
s=StandardScaler()
trainx=s.fit_transform(trainx)
testx=s.transform(testx)

ValueError: could not convert string to float: 'Vanessa Williams'